# **Laboratorio 8: Ready, Set, Deploy! 👩‍🚀👨‍🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Acuña
- Nombre de alumno 2: Tomás Ramírez

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/Diego-Acuna/mds-laboratorios.git)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

In [ ]:
# Importamos las librerías necesarias
import os
import json
import pickle
from datetime import datetime

import optuna
from optuna.samplers import TPESampler
import plotly.io as pio

import xgboost as xgb
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Definimos una función para recuperar el mejor modelo
def get_best_model(experiment_name: str):
    exp_local = mlflow.get_experiment_by_name(experiment_name)
    if exp_local is None:
        raise RuntimeError(f"Experimento '{experiment_name}' no encontrado en MLflow")
    runs = mlflow.search_runs(experiment_ids=[exp_local.experiment_id])
    if runs.empty or "metrics.valid_f1" not in runs.columns:
        raise RuntimeError("No se encontraron runs con la métrica 'valid_f1' en el experimento")
    best_run_id = runs.sort_values("metrics.valid_f1", ascending=False)["run_id"].iloc[0]
    return mlflow.sklearn.load_model(f"runs:/{best_run_id}/model")

def optimize_model(n_trials: int = 50, random_state: int = 42):
    """
    1) Optimiza hiperparámetros de XGBoost usando Optuna.
    2) Registra cada trial como un run dentro de un experimento nuevo (nombres interpretables).
       Cada run registra la métrica 'valid_f1' y guarda el modelo con mlflow.sklearn.log_model.
    3) Guarda los gráficos de Optuna en plots/ y los sube a MLflow como artefactos.
    4) Recupera el mejor modelo (get_best_model), lo serializa con pickle en models/model.pkl
       y guarda ese pickle en los artefactos MLflow en 'models'.
    5) Registra versiones de librerías y guarda configuración final y la importancia de variables
       en plots/.
    Devuelve (experiment_name, best_model_loaded).
    """

    # Cargamos los datos
    df = pd.read_csv("water_potability.csv")
    X = df.drop("Potability", axis=1)
    y = df["Potability"]

    # Separamos los datos en train, val y test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.2, random_state=random_state, stratify=y_train_val
    )

    # Creamos un nuevo experimento en MLflow
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"XGBoost_Optuna_{timestamp}"
    mlflow.set_experiment(experiment_name)
    exp = mlflow.get_experiment_by_name(experiment_name)

    # carpetas locales para artefactos
    os.makedirs("plots", exist_ok=True)
    os.makedirs("models", exist_ok=True)

    # Definimos la función objetivo para Optuna
    def objective(trial):
        # Definimos el espacio de búsqueda de hiperparámetros
        params = {
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 50, 400),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        }

        # Definimos el nombre de la run
        run_name = f"XGBoost_lr{params['learning_rate']:.3f}_md{params['max_depth']}_n{params['n_estimators']}"
        
        # Iniciamos una nueva run en MLflow
        with mlflow.start_run(run_name=run_name):
            # Registramos los hiperparámetros
            mlflow.log_params(params)

            # Entrenamos el modelo
            clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", **params)
            clf.fit(
                X_train,
                y_train,
                eval_set=[(X_val, y_val)],
                verbose=False,
            )

            preds = clf.predict(X_val)
            f1 = float(f1_score(y_val, preds))

            # Registramos la métrica de validación con el nombre requerido
            mlflow.log_metric("valid_f1", f1)

            # Registramos modelo en esta run para poder recuperarlo más tarde con runs:/<id>/model
            mlflow.sklearn.log_model(clf, artifact_path="model")

            # Guardamos la información del trial en un archivo JSON en plots/ tal como se pide
            trial_info = {"trial_number": trial.number, "valid_f1": f1, "params": params}
            trial_fn = f"plots/trial_{trial.number:03d}_info.json"
            with open(trial_fn, "w") as fh:
                json.dump(trial_info, fh)
            mlflow.log_artifact(trial_fn, artifact_path="plots")

        return f1

    # Ejecutamos la optimización con Optuna
    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=random_state))
    study.optimize(objective, n_trials=n_trials, n_jobs=1)

    # Guardamos los gráficos de Optuna en plots/ y los subimos a MLflow
    fig_hist = optuna.visualization.plot_optimization_history(study)
    pio.write_html(fig_hist, "plots/optuna_history.html", auto_open=False)

    fig_param = optuna.visualization.plot_param_importances(study)
    if fig_param is not None:
        pio.write_html(fig_param, "plots/optuna_param_importances.html", auto_open=False)

    # Guardamos los mejores hiperparámetros en un archivo JSON en plots/
    best_params = study.best_params
    with open("plots/best_params.json", "w") as fh:
        json.dump(best_params, fh)

    # Recuperamos el mejor modelo usando la función definida
    best_model_loaded = get_best_model(experiment_name)

    # Serializamos el modelo con pickle en models/model.pkl
    model_pickle_path = "models/model.pkl"
    with open(model_pickle_path, "wb") as fh:
        pickle.dump(best_model_loaded, fh)

    # Creamos una nueva run para guardar artefactos y metadata final
    final_run_name = f"artifacts_and_metadata_{timestamp}"
    with mlflow.start_run(run_name=final_run_name):
        # Guardamos las versiones de las librerías usadas
        mlflow.log_param("xgboost_version", xgb.__version__)
        mlflow.log_param("optuna_version", optuna.__version__)
        mlflow.log_param("mlflow_version", mlflow.__version__)
        mlflow.log_param("sklearn_version", sklearn.__version__)
        mlflow.log_param("pandas_version", pd.__version__)

        # Subimos los gráficos y archivos JSON generados en plots/
        mlflow.log_artifact("plots/optuna_history.html", artifact_path="plots")
        if os.path.exists("plots/optuna_param_importances.html"):
            mlflow.log_artifact("plots/optuna_param_importances.html", artifact_path="plots")
        mlflow.log_artifact("plots/best_params.json", artifact_path="plots")
        # subir todos los trial jsons generados
        for fn in sorted(os.listdir("plots")):
            if fn.startswith("trial_") and fn.endswith("_info.json"):
                mlflow.log_artifact(os.path.join("plots", fn), artifact_path="plots")

        # Importamos la importancia de variables y la guardamos en plots/
        try:
            if hasattr(best_model_loaded, "get_booster"):
                booster = best_model_loaded.get_booster()
                ax = xgb.plot_importance(booster, max_num_features=20)
            else:
                ax = xgb.plot_importance(best_model_loaded, max_num_features=20)
            plt.tight_layout()
            imp_path = "plots/importance_plot.png"
            plt.savefig(imp_path)
            plt.close()
            mlflow.log_artifact(imp_path, artifact_path="plots")
        except Exception:
            pass

        # Subimos el pickle del modelo
        mlflow.log_artifact(model_pickle_path, artifact_path="models")

    # Retornamos el nombre del experimento y el mejor modelo cargado
    print(f"Experiment saved: {experiment_name} (id={exp.experiment_id})")
    return experiment_name, best_model_loaded

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

**Respuestas:**

**Pregunta 4:** A continuación se muestran unos pantallazos que muestran la ejecución del contenedor y los resultados obtenidos. En primer lugar, se muestra una imagen con el estado del proyecto antes de inicializar la imagen del Docker de la aplicación:
<p align="center">
  <img src="images/00_project_tree.png" width="400">
</p>
A continuación, se muestra una imagen del comando para construir el Docker junto al resultado obtenido:
<p align="center">
  <img src="images/01_docker_build.png" width="1000">
</p>
Similarmente, se muestra el comando para obtener la imagen del Docker junto al resultado obtenido (que muestra la imagen y su tamaño).
<p align="center">
  <img src="images/02_docker_images.png" width="1000">
</p>
Por otro lado, se muestra el resultado obtenido al hacer la ejecución del contenedor con el comando run.
<p align="center">
  <img src="images/03_docker_run.png" width="1000">
</p>
De forma parecida a lo anterior, se muestran también los contenedores en ejecución con el comando docker ps.
<p align="center">
  <img src="images/04_docker_ps.png" width="1000">
</p>
También se muestra a continuación los logs del contenedor.
<p align="center">
  <img src="images/05_docker_logs.png" width="1000">
</p>
Además de comandos como tal, también se muestra a continuación cómo reacciona la API frente al input del enunciado.
<p align="center">
  <img src="images/06_post_response.png" width="1000">
</p>
Finalmente, se muestra el uso de recursos para ver si el contendor es liviano o no y así poder responder la siguiente pregunta.
<p align="center">
  <img src="images/07_docker_stats.png" width="1000">
</p>

**Pregunta 5:** En la revisión de los recursos utilizados por el contenedor, a partir del comando "docker stats --no-stream potability_api", se observa un consumo de CPU del 0.12 % y una utilización de memoria de aproximadamente 127.9 MiB, lo que representa solo un 1.6 % del total disponible. Estos valores evidencian que, una vez cargado el modelo y en funcionamiento el servidor Uvicorn, el contenedor mantiene un consumo muy bajo de recursos, utilizando procesamiento solo cuando recibe solicitudes. El uso de red y disco también es mínimo (unos pocos KB transferidos y cerca de 64 MB en operaciones de lectura, correspondientes principalmente a la carga del modelo y generación de logs), y el número de procesos activos (67 en total) es normal para un entorno Python que ejecuta servicios como FastAPI y XGBoost.


## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

1. ¿Cómo se diferencia Docker de una máquina virtual (VM)?
2. ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
3. ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
4. ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
5. ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

**Respuestas:**

**Pregunta 1:** Docker y las máquinas virtuales comparten la idea de aislar aplicaciones, pero lo hacen de forma muy distinta. Docker ofrece un aislamiento a nivel de proceso: los contenedores comparten el kernel del sistema operativo del host y utilizan espacios de nombres y control groups para mantener la separación. En cambio, una máquina virtual aísla a nivel de hardware, simulando componentes físicos y ejecutando un sistema operativo completo. Esta diferencia hace que Docker sea mucho más liviano, ya que no requiere emular hardware ni cargar otro sistema operativo. Por lo mismo, los contenedores se inician en segundos, mientras que una máquina virtual puede tardar varios minutos. En términos de eficiencia, los contenedores usan menos recursos, lo que permite ejecutar más servicios en la misma máquina física, aunque las máquinas virtuales ofrecen un nivel de aislamiento más fuerte y son más adecuadas cuando se necesita seguridad extrema o diferentes kernels.

**Pregunta 2:** Ejecutar una aplicación dentro de Docker ofrece varias ventajas frente a hacerlo directamente en el entorno local. Cuando se usa Docker, todas las dependencias, versiones de librerías, configuraciones y comandos necesarios quedan encapsulados en la imagen, lo que garantiza que el entorno sea exactamente el mismo en cualquier máquina donde se ejecute. Esto elimina los clásicos problemas de compatibilidad entre sistemas operativos y versiones, asegurando que lo que funciona en un computador funcionará igual en otro. En cambio, al ejecutar la aplicación localmente, el funcionamiento depende del entorno del usuario: versiones distintas de Python, librerías instaladas globalmente o diferencias en la configuración del sistema pueden provocar errores. Docker además proporciona aislamiento, evitando que los paquetes o dependencias de un proyecto interfieran con otros, y facilita la portabilidad, ya que basta con transferir la imagen y ejecutarla con los mismos comandos.

**Pregunta 3:** Docker asegura la consistencia entre entornos mediante el uso de imágenes inmutables, versionadas y reproducibles. Cada imagen se construye a partir de un Dockerfile, un documento que especifica paso a paso cómo crear el entorno: desde la base del sistema operativo hasta la instalación de dependencias y la configuración final. De este modo, cualquier persona o servidor que ejecute el comando docker build a partir de ese archivo obtendrá una imagen idéntica. Además, el uso de archivos como el archivo requirements.txt creado para este laboratorio, permite fijar las versiones exactas de las librerías, asegurando que no haya diferencias entre desarrollo, prueba y producción. En resumen, Docker crea entornos totalmente replicables y previsibles, evitando los errores derivados de configuraciones distintas entre máquinas.

**Pregunta 4:** Docker maneja la persistencia de datos a través de volúmenes, que permiten que la información generada dentro del contenedor no se pierda al detener o eliminarlo. Los volúmenes pueden ser de dos tipos: los bind mounts y los named volumes. En los primeros, se vincula una carpeta del sistema host con una ruta dentro del contenedor, lo que permite acceder directamente a los archivos desde ambos entornos. Los named volumes, en cambio, son gestionados por Docker y almacenan los datos de forma independiente del sistema de archivos del host, lo que es útil en despliegues más controlados o compartidos. En ambos casos, los datos persisten incluso si el contenedor se elimina, garantizando que los resultados, modelos entrenados o configuraciones no se pierdan entre ejecuciones. Este mecanismo también facilita la separación entre el código (que puede actualizarse libremente) y la información almacenada (que debe preservarse). En resumen, los volúmenes son una herramienta clave para asegurar la persistencia y la interoperabilidad entre el entorno de desarrollo y el contenedor.

**Pregunta 5:** Dockerfile, por un lado, es la receta para construir una imagen. Define base (en el caso de este laboratorio python:3.11-slim), copia fuentes, instala dependencias, expone puertos y comando final (Uvicorn), obteniendo como resultado una imagen portable (en este caso, generamos una imagen de aproximadamente 3GB). 

Por otro lado, docker-compose.yml es una orquestación en tiempo de ejecución para uno o varios servicios (app, DB, broker). Declara volúmenes, puertos, redes, variables de forma declarativa y levanta todo con docker compose up -d.

# Conclusión

Éxito!
<div align="center">
  <img src="https://i.pinimg.com/originals/55/f5/fd/55f5fdc9455989f8caf7fca7f93bd96a.gif" width="500">
</div>